# Processamento de Dados Raster

Se você já usou o Spatial Analyst do ArcGIS ou o GRASS GIS, aqui vamos fazer operações similares com código! É como ter uma calculadora raster programável para fazer análises complexas de imagens e MDTs.

## Objetivos
- Realizar álgebra de mapas (como Map Algebra no ArcGIS)
- Aplicar filtros e transformações (como realce de imagens)
- Executar análises de terreno (como análise de relevo)
- Trabalhar com múltiplas bandas (como composições RGB)

## 1. Configuração do Ambiente

Vamos preparar nosso laboratório de processamento digital:

In [ ]:
# Nossa estação de processamento avançado:
import rasterio               # Nossa ferramenta principal de raster
import numpy as np            # Nossa calculadora matricial
import matplotlib.pyplot as plt  # Nossa ferramenta de visualização
from rasterio.plot import show   # Para mostrar rasters
from scipy import ndimage      # Para filtros espaciais
from rasterio.windows import Window  # Para processar por partes
from rasterio.enums import Resampling  # Para reamostragem

plt.rcParams['figure.figsize'] = (12, 8)

## 2. Álgebra de Mapas

### 2.1 Operações Básicas

Álgebra de mapas é como fazer contas com camadas raster inteiras:

In [ ]:
# Criando rasters de teste
# Como gerar imagens sintéticas para experimentos
def criar_raster_teste(output_path, pattern='gradient'):
    height, width = 100, 100
    if pattern == 'gradient':
        # Como um gradiente de elevação
        data = np.linspace(0, 1, height * width).reshape((height, width))
    else:  # pattern == 'random'
        # Como uma superfície irregular
        data = np.random.rand(height, width)
    
    # Salvando o raster
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=data.dtype,
        crs='EPSG:4326',
        transform=rasterio.transform.from_origin(-46.65, -23.55, 0.01, 0.01)
    ) as dst:
        dst.write(data, 1)

# Criando dois rasters diferentes
criar_raster_teste('../data/processed/raster1.tif', 'gradient')  # Como um MDT
criar_raster_teste('../data/processed/raster2.tif', 'random')    # Como uma imagem de satélite

# Lendo os rasters para processamento
with rasterio.open('../data/processed/raster1.tif') as src1, \
     rasterio.open('../data/processed/raster2.tif') as src2:
    data1 = src1.read(1)
    data2 = src2.read(1)
    profile = src1.profile
    
    # Operações de álgebra de mapas
    soma = data1 + data2          # Como somar duas superfícies
    diferenca = data1 - data2     # Como calcular diferenças
    produto = data1 * data2       # Como multiplicar camadas
    divisao = np.divide(data1, data2, where=data2!=0)  # Como fazer razões
    
    # Visualização dos resultados
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 12))
    
    im1 = ax1.imshow(soma)
    ax1.set_title('Soma (como combinar camadas)')
    plt.colorbar(im1, ax=ax1)
    
    im2 = ax2.imshow(diferenca)
    ax2.set_title('Diferença (como detectar mudanças)')
    plt.colorbar(im2, ax=ax2)
    
    im3 = ax3.imshow(produto)
    ax3.set_title('Produto (como pesos)')
    plt.colorbar(im3, ax=ax3)
    
    im4 = ax4.imshow(divisao)
    ax4.set_title('Divisão (como índices)')
    plt.colorbar(im4, ax=ax4)
    
    plt.tight_layout()
    plt.show()

## 3. Filtros e Transformações

### 3.1 Filtros de Convolução

Filtros são como passar uma lente especial sobre a imagem:

In [ ]:
# Aplicando diferentes filtros
with rasterio.open('../data/processed/raster1.tif') as src:
    data = src.read(1)
    profile = src.profile
    
    # Definindo kernels (como diferentes lentes)
    kernel_suavizacao = np.array([[1, 1, 1],    # Como desfoque
                                  [1, 1, 1],
                                  [1, 1, 1]]) / 9.0
    
    kernel_sobel_x = np.array([[-1, 0, 1],     # Como detector de bordas X
                              [-2, 0, 2],
                              [-1, 0, 1]])
    
    kernel_sobel_y = np.array([[-1, -2, -1],   # Como detector de bordas Y
                              [0, 0, 0],
                              [1, 2, 1]])
    
    # Aplicando os filtros
    suavizado = ndimage.convolve(data, kernel_suavizacao)  # Suavização
    gradiente_x = ndimage.convolve(data, kernel_sobel_x)   # Gradiente X
    gradiente_y = ndimage.convolve(data, kernel_sobel_y)   # Gradiente Y
    magnitude_gradiente = np.sqrt(gradiente_x**2 + gradiente_y**2)  # Magnitude
    
    # Visualização
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 12))
    
    im1 = ax1.imshow(data)
    ax1.set_title('Original')
    plt.colorbar(im1, ax=ax1)
    
    im2 = ax2.imshow(suavizado)
    ax2.set_title('Suavizado (como média móvel)')
    plt.colorbar(im2, ax=ax2)
    
    im3 = ax3.imshow(gradiente_x)
    ax3.set_title('Gradiente X (como declividade)')
    plt.colorbar(im3, ax=ax3)
    
    im4 = ax4.imshow(magnitude_gradiente)
    ax4.set_title('Magnitude do Gradiente (como relevo)')
    plt.colorbar(im4, ax=ax4)
    
    plt.tight_layout()
    plt.show()

## 4. Análise de Terreno

### 4.1 Cálculo de Declividade e Aspecto

Como extrair informações de relevo de um MDT:

In [ ]:
def calcular_declividade(dem, resolucao_x, resolucao_y):
    """Calcula a declividade (como inclinação do terreno)"""
    dx = ndimage.sobel(dem, axis=1) / (8.0 * resolucao_x)
    dy = ndimage.sobel(dem, axis=0) / (8.0 * resolucao_y)
    declividade = np.degrees(np.arctan(np.sqrt(dx**2 + dy**2)))
    return declividade

def calcular_aspecto(dem, resolucao_x, resolucao_y):
    """Calcula o aspecto (como orientação da vertente)"""
    dx = ndimage.sobel(dem, axis=1) / (8.0 * resolucao_x)
    dy = ndimage.sobel(dem, axis=0) / (8.0 * resolucao_y)
    aspecto = np.degrees(np.arctan2(dy, dx))
    return aspecto

# Criando um MDT sintético
# Como simular um terreno ondulado
x = np.linspace(0, 100, 100)
y = np.linspace(0, 100, 100)
X, Y = np.meshgrid(x, y)
Z = 1000 + 100 * np.sin(X/10) * np.cos(Y/10)  # Elevação sintética

# Calculando declividade e aspecto
declividade = calcular_declividade(Z, 1, 1)
aspecto = calcular_aspecto(Z, 1, 1)

# Visualização
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

im1 = ax1.imshow(Z, cmap='terrain')
ax1.set_title('Elevação (MDT)')
plt.colorbar(im1, ax=ax1)

im2 = ax2.imshow(declividade, cmap='YlOrRd')
ax2.set_title('Declividade (graus)')
plt.colorbar(im2, ax=ax2)

im3 = ax3.imshow(aspecto, cmap='hsv')
ax3.set_title('Aspecto (orientação)')
plt.colorbar(im3, ax=ax3)

plt.tight_layout()
plt.show()

## 5. Processamento de Múltiplas Bandas

Como trabalhar com imagens multiespectrais:

In [ ]:
# Criando um raster RGB
# Como simular uma imagem multiespectral
def criar_raster_rgb(output_path):
    height, width = 100, 100
    data = np.zeros((3, height, width))
    
    # Criando padrões para cada banda
    data[0] = np.random.rand(height, width)  # Banda do vermelho
    data[1] = np.linspace(0, 1, height*width).reshape((height, width))  # Verde
    data[2] = np.sin(np.linspace(0, 10, height*width)).reshape((height, width))  # Azul
    
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=3,
        dtype=data.dtype,
        crs='EPSG:4326',
        transform=rasterio.transform.from_origin(-46.65, -23.55, 0.01, 0.01)
    ) as dst:
        dst.write(data)

# Criando e processando a imagem RGB
criar_raster_rgb('../data/processed/rgb.tif')

with rasterio.open('../data/processed/rgb.tif') as src:
    rgb = src.read()
    
    # Calculando NDVI simulado
    # Como calcular índices de vegetação
    ndvi = (rgb[1] - rgb[0]) / (rgb[1] + rgb[0])
    
    # Visualização
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Composição RGB
    rgb_plot = np.moveaxis(rgb, 0, -1)  # Ajustando dimensões
    ax1.imshow(rgb_plot)
    ax1.set_title('Composição RGB (cor verdadeira)')
    
    # NDVI
    im2 = ax2.imshow(ndvi, cmap='RdYlGn')
    ax2.set_title('NDVI Simulado (índice de vegetação)')
    plt.colorbar(im2, ax=ax2)
    
    plt.tight_layout()
    plt.show()

## 6. Exercícios Práticos

Vamos praticar como um analista de sensoriamento remoto:

1. Análise de Terreno:
   - Use um MDT real da sua região
   - Calcule declividade e aspecto
   - Identifique áreas de risco
   - Faça um mapa de exposição solar

2. Processamento de Imagens:
   - Use uma imagem Landsat ou Sentinel
   - Calcule índices (NDVI, NDWI)
   - Aplique realce de contraste
   - Compare os resultados

3. Álgebra de Mapas:
   - Combine diferentes rasters
   - Crie um modelo de risco
   - Faça reclassificação
   - Visualize os resultados